In [6]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim

In [3]:
# Paths
methods = ["classic", "neSyAI", "neural"]
base_dir = "/Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/finalPercepts"  # folder containing classic/, neSyAI/, neural/
video_names = ["night1_percept.mp4", "North-East_percept.mp4", "pplWalk1_percept.mp4", "pplWalk2_percept.mp4", "road-video-dubai_percept.mp4"]

def compute_iou(mask1, mask2):
    mask1_bin = mask1 > 0
    mask2_bin = mask2 > 0
    intersection = np.logical_and(mask1_bin, mask2_bin).sum()
    union = np.logical_or(mask1_bin, mask2_bin).sum()
    return intersection / union if union != 0 else 0

def compute_dice(mask1, mask2):
    mask1_bin = mask1 > 0
    mask2_bin = mask2 > 0
    intersection = np.logical_and(mask1_bin, mask2_bin).sum()
    return (2 * intersection) / (mask1_bin.sum() + mask2_bin.sum() + 1e-8)

def compute_ssim_frame(mask1, mask2):
    # Convert to grayscale
    if len(mask1.shape) == 3:
        mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2GRAY)
    if len(mask2.shape) == 3:
        mask2 = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY)
    return ssim(mask1, mask2)

def compute_psnr(mask1, mask2):
    return cv2.PSNR(mask1, mask2)

# comparison loop
results = {}

for video_file in video_names:
    video_base = os.path.splitext(video_file)[0]
    results[video_base] = {}
    
    # open video capture
    caps = {method: cv2.VideoCapture(os.path.join(base_dir, method, video_file)) for method in methods}
    
    frame_count = int(min(caps[m].get(cv2.CAP_PROP_FRAME_COUNT) for m in methods))
    
    # initialize accumulators
    accum_metrics = {}
    for i in range(len(methods)):
        for j in range(i + 1, len(methods)):
            pair = f"{methods[i]}_vs_{methods[j]}"
            accum_metrics[pair] = {"IoU": 0, "Dice": 0, "SSIM": 0, "PSNR": 0}
    
    # process each frame
    for _ in range(frame_count):
        frames = {m: caps[m].read()[1] for m in methods}
        
        # Skip if any frame failed to read
        if any(f is None for f in frames.values()):
            continue
        
        # resize all frames to the same size 
        min_height = min(f.shape[0] for f in frames.values())
        min_width = min(f.shape[1] for f in frames.values())
        for m in methods:
            frames[m] = cv2.resize(frames[m], (min_width, min_height))
        
        # Compute metrics for each pair
        for i in range(len(methods)):
            for j in range(i + 1, len(methods)):
                m1, m2 = methods[i], methods[j]
                pair = f"{m1}_vs_{m2}"
                accum_metrics[pair]["IoU"] += compute_iou(frames[m1], frames[m2])
                accum_metrics[pair]["Dice"] += compute_dice(frames[m1], frames[m2])
                accum_metrics[pair]["SSIM"] += compute_ssim_frame(frames[m1], frames[m2])
                accum_metrics[pair]["PSNR"] += compute_psnr(frames[m1], frames[m2])
    
    # average metrics over frames
    for pair, metrics in accum_metrics.items():
        results[video_base][pair] = {k: v / frame_count for k, v in metrics.items()}
    
    # release video captures
    for cap in caps.values():
        cap.release()

# print results
for video, comps in results.items():
    print(f"\nResults for {video}:")
    for pair, metrics in comps.items():
        print(f"{pair}: IoU={metrics['IoU']:.3f}, Dice={metrics['Dice']:.3f}, SSIM={metrics['SSIM']:.3f}, PSNR={metrics['PSNR']:.2f}")


Results for night1_percept:
classic_vs_neSyAI: IoU=0.466, Dice=0.630, SSIM=0.286, PSNR=9.40
classic_vs_neural: IoU=0.937, Dice=0.968, SSIM=0.452, PSNR=10.61
neSyAI_vs_neural: IoU=0.468, Dice=0.632, SSIM=0.287, PSNR=11.60

Results for North-East_percept:
classic_vs_neSyAI: IoU=0.171, Dice=0.291, SSIM=0.195, PSNR=6.45
classic_vs_neural: IoU=0.947, Dice=0.973, SSIM=0.643, PSNR=15.17
neSyAI_vs_neural: IoU=0.169, Dice=0.288, SSIM=0.196, PSNR=5.07

Results for pplWalk1_percept:
classic_vs_neSyAI: IoU=0.489, Dice=0.655, SSIM=0.221, PSNR=8.81
classic_vs_neural: IoU=0.902, Dice=0.948, SSIM=0.373, PSNR=11.11
neSyAI_vs_neural: IoU=0.527, Dice=0.688, SSIM=0.416, PSNR=12.20

Results for pplWalk2_percept:
classic_vs_neSyAI: IoU=0.446, Dice=0.616, SSIM=0.207, PSNR=7.15
classic_vs_neural: IoU=0.895, Dice=0.945, SSIM=0.348, PSNR=8.39
neSyAI_vs_neural: IoU=0.473, Dice=0.641, SSIM=0.368, PSNR=11.00

Results for road-video-dubai_percept:
classic_vs_neSyAI: IoU=0.212, Dice=0.346, SSIM=0.249, PSNR=7.65
cla

In [10]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim

# -----------------------------
# PATHS
# -----------------------------
methods = ["classic", "neSyAI", "neural"]

original_video_dir = "/Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/Videos"
base_dir = "/Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/finalPercepts"

video_names = [
    "night1.mp4",
    "North-East.mp4",
    "pplWalk1.mp4",
    "pplWalk2.mp4",
    "road-video-dubai.mp4"
]

percept_names = [
    "night1_percept.mp4",
    "North-East_percept.mp4",
    "pplWalk1_percept.mp4",
    "pplWalk2_percept.mp4",
    "road-video-dubai_percept.mp4"
]

# -----------------------------
# METRICS
# -----------------------------
def compute_iou(mask1, mask2):
    mask1_bin = mask1 > 0
    mask2_bin = mask2 > 0
    intersection = np.logical_and(mask1_bin, mask2_bin).sum()
    union = np.logical_or(mask1_bin, mask2_bin).sum()
    return intersection / union if union != 0 else 0

def compute_dice(mask1, mask2):
    mask1_bin = mask1 > 0
    mask2_bin = mask2 > 0
    intersection = np.logical_and(mask1_bin, mask2_bin).sum()
    return (2 * intersection) / (mask1_bin.sum() + mask2_bin.sum() + 1e-8)

def compute_ssim_frame(img1, img2):
    if len(img1.shape) == 3:
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    if len(img2.shape) == 3:
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    return ssim(img1, img2)

def compute_psnr(img1, img2):
    return cv2.PSNR(img1, img2)

def boundary_f1(mask1, mask2):
    if len(mask1.shape) == 3:
        mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2GRAY)
    if len(mask2.shape) == 3:
        mask2 = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY)

    edge1 = cv2.Canny(mask1, 50, 150) > 0
    edge2 = cv2.Canny(mask2, 50, 150) > 0

    tp = np.logical_and(edge1, edge2).sum()

    precision = tp / (edge1.sum() + 1e-8)
    recall = tp / (edge2.sum() + 1e-8)

    if precision + recall == 0:
        return 0
    return 2 * precision * recall / (precision + recall)

def temporal_consistency(prev_frame, curr_frame):
    if len(prev_frame.shape) == 3:
        prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    if len(curr_frame.shape) == 3:
        curr_frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    return ssim(prev_frame, curr_frame)

# -----------------------------
# MAIN LOOP
# -----------------------------
for video_file, percept_file in zip(video_names, percept_names):

    print(f"\n===== {video_file} =====")

    caps = {}

    for m in methods:
        path = os.path.join(base_dir, m, percept_file)
        if not os.path.exists(path):
            print(f"❌ Missing file: {path}")
            continue
        caps[m] = cv2.VideoCapture(path)

    orig_path = os.path.join(original_video_dir, video_file)
    if not os.path.exists(orig_path):
        print(f"❌ Missing original: {orig_path}")
        continue

    original_cap = cv2.VideoCapture(orig_path)

    prev_frames = {m: None for m in methods}
    temporal_scores = {m: 0 for m in methods}

    accum = {}
    frame_counter = 0

    all_sources = methods + ["original"]
    for i in range(len(all_sources)):
        for j in range(i + 1, len(all_sources)):
            pair = f"{all_sources[i]}_vs_{all_sources[j]}"
            accum[pair] = {
                "IoU":0,"Dice":0,"SSIM":0,
                "PSNR":0,"BoundaryF1":0
            }

    # FRAME LOOP
    while True:

        frames = {}

        for m in methods:
            ret, frame = caps[m].read()
            if not ret:
                frames = None
                break
            frames[m] = frame

        ret, orig_frame = original_cap.read()
        if not ret or frames is None:
            break

        frames["original"] = orig_frame

        min_h = min(f.shape[0] for f in frames.values())
        min_w = min(f.shape[1] for f in frames.values())

        for k in frames:
            frames[k] = cv2.resize(frames[k], (min_w, min_h))

        # temporal
        for m in methods:
            if prev_frames[m] is not None:
                temporal_scores[m] += temporal_consistency(prev_frames[m], frames[m])
            prev_frames[m] = frames[m]

        # pair metrics
        keys = list(frames.keys())
        for i in range(len(keys)):
            for j in range(i+1, len(keys)):
                k1, k2 = keys[i], keys[j]
                pair = f"{k1}_vs_{k2}"

                accum[pair]["IoU"] += compute_iou(frames[k1], frames[k2])
                accum[pair]["Dice"] += compute_dice(frames[k1], frames[k2])
                accum[pair]["SSIM"] += compute_ssim_frame(frames[k1], frames[k2])
                accum[pair]["PSNR"] += compute_psnr(frames[k1], frames[k2])
                accum[pair]["BoundaryF1"] += boundary_f1(frames[k1], frames[k2])

        frame_counter += 1

    if frame_counter == 0:
        print("❌ No frames processed")
        continue

    # RESULTS
    for pair in accum:
        print(f"\n{pair}")
        for metric in accum[pair]:
            print(f"{metric}: {accum[pair][metric]/frame_counter:.3f}")

    print("\nTemporal Consistency:")
    for m in temporal_scores:
        print(f"{m}: {temporal_scores[m]/frame_counter:.3f}")

    for cap in caps.values():
        cap.release()
    original_cap.release()



===== night1.mp4 =====

classic_vs_neSyAI
IoU: 0.466
Dice: 0.630
SSIM: 0.286
PSNR: 9.405
BoundaryF1: 0.036

classic_vs_neural
IoU: 0.937
Dice: 0.968
SSIM: 0.452
PSNR: 10.613
BoundaryF1: 0.056

classic_vs_original
IoU: 0.694
Dice: 0.819
SSIM: 0.100
PSNR: 10.248
BoundaryF1: 0.044

neSyAI_vs_neural
IoU: 0.468
Dice: 0.632
SSIM: 0.287
PSNR: 11.602
BoundaryF1: 0.077

neSyAI_vs_original
IoU: 0.335
Dice: 0.498
SSIM: 0.010
PSNR: 7.256
BoundaryF1: 0.046

neural_vs_original
IoU: 0.683
Dice: 0.811
SSIM: 0.051
PSNR: 7.846
BoundaryF1: 0.042

Temporal Consistency:
classic: 0.991
neSyAI: 0.934
neural: 0.883

===== North-East.mp4 =====

classic_vs_neSyAI
IoU: 0.171
Dice: 0.291
SSIM: 0.195
PSNR: 6.455
BoundaryF1: 0.007

classic_vs_neural
IoU: 0.947
Dice: 0.973
SSIM: 0.643
PSNR: 15.171
BoundaryF1: 0.143

classic_vs_original
IoU: 0.717
Dice: 0.835
SSIM: 0.140
PSNR: 7.775
BoundaryF1: 0.052

neSyAI_vs_neural
IoU: 0.169
Dice: 0.288
SSIM: 0.196
PSNR: 5.067
BoundaryF1: 0.025

neSyAI_vs_original
IoU: 0.123
Dic

In [1]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import normalized_mutual_info_score

# -----------------------------
# PATHS
# -----------------------------
methods = ["classic", "neSyAI", "neural"]

original_video_dir = "/Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/Videos"
base_dir = "/Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/finalPercepts"

video_names = [
    "night1.mp4",
    "North-East.mp4",
    "pplWalk1.mp4",
    "pplWalk2.mp4",
    "road-video-dubai.mp4"
]

percept_names = [
    "night1_percept.mp4",
    "North-East_percept.mp4",
    "pplWalk1_percept.mp4",
    "pplWalk2_percept.mp4",
    "road-video-dubai_percept.mp4"
]

# -----------------------------
# METRICS
# -----------------------------
def compute_iou(mask1, mask2):
    mask1_bin = mask1 > 0
    mask2_bin = mask2 > 0
    intersection = np.logical_and(mask1_bin, mask2_bin).sum()
    union = np.logical_or(mask1_bin, mask2_bin).sum()
    return intersection / union if union != 0 else 0

def compute_dice(mask1, mask2):
    mask1_bin = mask1 > 0
    mask2_bin = mask2 > 0
    intersection = np.logical_and(mask1_bin, mask2_bin).sum()
    return (2 * intersection) / (mask1_bin.sum() + mask2_bin.sum() + 1e-8)

def compute_ssim_frame(img1, img2):
    if len(img1.shape) == 3:
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    if len(img2.shape) == 3:
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    return ssim(img1, img2)

def compute_psnr(img1, img2):
    return cv2.PSNR(img1, img2)

def boundary_f1(mask1, mask2):
    if len(mask1.shape) == 3:
        mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2GRAY)
    if len(mask2.shape) == 3:
        mask2 = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY)

    edge1 = cv2.Canny(mask1, 50, 150) > 0
    edge2 = cv2.Canny(mask2, 50, 150) > 0

    tp = np.logical_and(edge1, edge2).sum()
    precision = tp / (edge1.sum() + 1e-8)
    recall = tp / (edge2.sum() + 1e-8)

    if precision + recall == 0:
        return 0
    return 2 * precision * recall / (precision + recall)

def temporal_consistency(prev_frame, curr_frame):
    if len(prev_frame.shape) == 3:
        prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    if len(curr_frame.shape) == 3:
        curr_frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    return ssim(prev_frame, curr_frame)

def compute_nmi(mask1, mask2):
    # Flatten masks for categorical comparison
    return normalized_mutual_info_score(mask1.flatten(), mask2.flatten())

# -----------------------------
# MAIN LOOP
# -----------------------------
for video_file, percept_file in zip(video_names, percept_names):

    print(f"\n===== {video_file} =====")

    caps = {}

    for m in methods:
        path = os.path.join(base_dir, m, percept_file)
        if not os.path.exists(path):
            print(f"❌ Missing file: {path}")
            continue
        caps[m] = cv2.VideoCapture(path)

    orig_path = os.path.join(original_video_dir, video_file)
    if not os.path.exists(orig_path):
        print(f"❌ Missing original: {orig_path}")
        continue

    original_cap = cv2.VideoCapture(orig_path)

    prev_frames = {m: None for m in methods}
    temporal_scores = {m: 0 for m in methods}

    accum = {}
    frame_counter = 0

    all_sources = methods + ["original"]
    for i in range(len(all_sources)):
        for j in range(i + 1, len(all_sources)):
            pair = f"{all_sources[i]}_vs_{all_sources[j]}"
            accum[pair] = {
                "IoU":0,"Dice":0,"SSIM":0,
                "PSNR":0,"BoundaryF1":0,"NMI":0
            }

    # FRAME LOOP
    while True:

        frames = {}

        for m in methods:
            ret, frame = caps[m].read()
            if not ret:
                frames = None
                break
            frames[m] = frame

        ret, orig_frame = original_cap.read()
        if not ret or frames is None:
            break

        frames["original"] = orig_frame

        min_h = min(f.shape[0] for f in frames.values())
        min_w = min(f.shape[1] for f in frames.values())

        for k in frames:
            frames[k] = cv2.resize(frames[k], (min_w, min_h))

        # temporal
        for m in methods:
            if prev_frames[m] is not None:
                temporal_scores[m] += temporal_consistency(prev_frames[m], frames[m])
            prev_frames[m] = frames[m]

        # pair metrics
        keys = list(frames.keys())
        for i in range(len(keys)):
            for j in range(i+1, len(keys)):
                k1, k2 = keys[i], keys[j]
                pair = f"{k1}_vs_{k2}"

                accum[pair]["IoU"] += compute_iou(frames[k1], frames[k2])
                accum[pair]["Dice"] += compute_dice(frames[k1], frames[k2])
                accum[pair]["SSIM"] += compute_ssim_frame(frames[k1], frames[k2])
                accum[pair]["PSNR"] += compute_psnr(frames[k1], frames[k2])
                accum[pair]["BoundaryF1"] += boundary_f1(frames[k1], frames[k2])
                accum[pair]["NMI"] += compute_nmi(frames[k1], frames[k2])

        frame_counter += 1

    if frame_counter == 0:
        print("❌ No frames processed")
        continue

    # RESULTS
    for pair in accum:
        print(f"\n{pair}")
        for metric in accum[pair]:
            print(f"{metric}: {accum[pair][metric]/frame_counter:.3f}")

    print("\nTemporal Consistency:")
    for m in temporal_scores:
        print(f"{m}: {temporal_scores[m]/frame_counter:.3f}")

    for cap in caps.values():
        cap.release()
    original_cap.release()



===== night1.mp4 =====

classic_vs_neSyAI
IoU: 0.466
Dice: 0.630
SSIM: 0.286
PSNR: 9.405
BoundaryF1: 0.036
NMI: 0.185

classic_vs_neural
IoU: 0.937
Dice: 0.968
SSIM: 0.452
PSNR: 10.613
BoundaryF1: 0.056
NMI: 0.322

classic_vs_original
IoU: 0.694
Dice: 0.819
SSIM: 0.100
PSNR: 10.248
BoundaryF1: 0.044
NMI: 0.110

neSyAI_vs_neural
IoU: 0.468
Dice: 0.632
SSIM: 0.287
PSNR: 11.602
BoundaryF1: 0.077
NMI: 0.233

neSyAI_vs_original
IoU: 0.335
Dice: 0.498
SSIM: 0.010
PSNR: 7.256
BoundaryF1: 0.046
NMI: 0.095

neural_vs_original
IoU: 0.683
Dice: 0.811
SSIM: 0.051
PSNR: 7.846
BoundaryF1: 0.042
NMI: 0.103

Temporal Consistency:
classic: 0.991
neSyAI: 0.934
neural: 0.883

===== North-East.mp4 =====

classic_vs_neSyAI
IoU: 0.171
Dice: 0.291
SSIM: 0.195
PSNR: 6.455
BoundaryF1: 0.007
NMI: 0.111

classic_vs_neural
IoU: 0.947
Dice: 0.973
SSIM: 0.643
PSNR: 15.171
BoundaryF1: 0.143
NMI: 0.393

classic_vs_original
IoU: 0.717
Dice: 0.835
SSIM: 0.140
PSNR: 7.775
BoundaryF1: 0.052
NMI: 0.120

neSyAI_vs_neural
